In [86]:
# base_path = './'

In [87]:
import pandas as pd
from time import time
import tensorflow as tf
import os

In [88]:
def gerar_rede(dir_path: str, num_classes: int, n_neurons: int, n_hidden_layers: int):
    data_train = pd.read_csv(dir_path + "\\" + "mnist_train.csv").to_numpy()
    data_test = pd.read_csv(dir_path + "\\" + "mnist_test.csv").to_numpy()

    x_train, y_train = data_train[:, :-1], data_train[:, -1]
    x_test, y_test = data_test[:, :-1], data_test[:, -1]

    y_train_ohe = tf.keras.utils.to_categorical(
        y_train, num_classes=num_classes)
    y_test_ohe = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

    model = tf.keras.Sequential(
        [
            tf.keras.layers.Input(shape=[x_train.shape[1]]),
        ]
    )

    for _ in range(n_hidden_layers):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))

    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    
    model_path = os.path.join(
        dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons.keras"
    )
    if not model_path:
        os.makedirs(os.path.join(dir_path, "models"))
    # if not os.path.exists(os.path.join(dir_path, "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons.h5")):
    #     os.makedirs(os.path.join(dir_path,  "models", f"model_{n_hidden_layers}layers_{n_neurons}neurons.h5"))
    

    es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
    ck = tf.keras.callbacks.ModelCheckpoint(
        f'{model_path}', monitor="val_accuracy", save_best_only=True
    )

    start = time()
    model.fit(
        x_train,
        y_train_ohe,
        batch_size=4,
        epochs=100,
        validation_data=(x_test, y_test_ohe),
        verbose=2,
        callbacks=[ck, es],
    )
    end = time() 
    
    print(f"Tempo de Treinamento: {end-start}")

    # salvar modelo
    model = tf.keras.models.load_model(model_path)

    # avaliar modelo com os dados de treinamento
    print("Resultado Treinamento")
    model.evaluate(x_train, y_train_ohe, verbose=2)

    # avaliar modelo com os dados de teste
    print("Resultado Teste")
    model.evaluate(x_test, y_test_ohe, verbose=2)

In [89]:
n_hidden_layers = 6
n_neurons = 20
num_classes = 10
dir_path = os.getcwd()
gerar_rede(dir_path, num_classes, n_neurons, n_hidden_layers)

Epoch 1/100
150/150 - 3s - 18ms/step - accuracy: 0.2733 - loss: 1.9819 - val_accuracy: 0.4100 - val_loss: 1.5340
Epoch 2/100
150/150 - 0s - 3ms/step - accuracy: 0.5467 - loss: 1.2849 - val_accuracy: 0.5500 - val_loss: 1.1633
Epoch 3/100
150/150 - 0s - 3ms/step - accuracy: 0.6933 - loss: 0.8455 - val_accuracy: 0.6400 - val_loss: 1.0436
Epoch 4/100
150/150 - 0s - 3ms/step - accuracy: 0.7600 - loss: 0.6658 - val_accuracy: 0.7000 - val_loss: 0.9010
Epoch 5/100
150/150 - 0s - 2ms/step - accuracy: 0.8417 - loss: 0.4869 - val_accuracy: 0.7600 - val_loss: 0.8124
Epoch 6/100
150/150 - 0s - 3ms/step - accuracy: 0.8600 - loss: 0.4010 - val_accuracy: 0.7700 - val_loss: 0.7770
Epoch 7/100
150/150 - 1s - 4ms/step - accuracy: 0.9133 - loss: 0.2718 - val_accuracy: 0.7500 - val_loss: 0.7420
Epoch 8/100
150/150 - 0s - 2ms/step - accuracy: 0.9033 - loss: 0.2960 - val_accuracy: 0.8300 - val_loss: 0.7967
Epoch 9/100
150/150 - 0s - 2ms/step - accuracy: 0.9383 - loss: 0.1948 - val_accuracy: 0.7900 - val_loss